In [43]:
!pip install pandas
!pip install cassandra-driver

In [44]:
from cassandra.cluster import Cluster
from cassandra.auth import PlainTextAuthProvider
import json

cloud_config= {
  'secure_connect_bundle': '/content/drive/MyDrive/Colab Notebooks/secure-connect-momin-db.zip'
}

with open("/content/drive/MyDrive/Colab Notebooks/momin_db-token.json") as f:
    secrets = json.load(f)

CLIENT_ID = secrets["clientId"]
CLIENT_SECRET = secrets["secret"]

auth_provider = PlainTextAuthProvider(CLIENT_ID, CLIENT_SECRET)
cluster = Cluster(cloud=cloud_config, auth_provider=auth_provider)
session = cluster.connect()

session.set_keyspace('sales1')
print(f"Connected to the Database and connected to the keyspace: {session.keyspace}")

Connected to the Database and connected to the keyspace: sales1


In [45]:
# Create Bronze Level Table
session.execute("""
    CREATE TABLE IF NOT EXISTS bronze_table (
        region TEXT,
        country TEXT,
        item_type TEXT,
        sales_channel TEXT,
        order_priority TEXT,
        order_date TEXT,
        order_id UUID PRIMARY KEY,
        ship_date TEXT,
        units_sold INT,
        unit_price DECIMAL,
        unit_cost DECIMAL,
        total_revenue DECIMAL,
        total_cost DECIMAL,
        total_profit DECIMAL
    );
""")
print("Bronze Level Table created successfully.")



Bronze Level Table created successfully.


In [46]:
import pandas as pd
import uuid

# Load data from CSV using pandas
csv_file_path = '/content/drive/MyDrive/Colab Notebooks/sales_100.csv'  # Replace with your actual CSV file path
df = pd.read_csv(csv_file_path)

# Inserting csv data into the Bronze table
for _, row in df.iterrows():
    session.execute("""
        INSERT INTO bronze_table (region, country, item_type, sales_channel, order_priority,
                                  order_date, order_id, ship_date, units_sold, unit_price,
                                  unit_cost, total_revenue, total_cost, total_profit)
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        row['Region'], row['Country'], row['Item Type'], row['Sales Channel'],
        row['Order Priority'], row['Order Date'], uuid.uuid4(),  # Generate UUID for each order
        row['Ship Date'], row['UnitsSold'], row['UnitPrice'],
        row['UnitCost'], row['TotalRevenue'], row['TotalCost'], row['TotalProfit']
    ))

print("Data inserted into bronze_table.")

data = session.execute("SELECT * FROM bronze_table LIMIT 5;")
for row in data:
    print(row)


Data inserted into bronze_table.
Row(order_id=UUID('7b1e856f-abdd-446d-b4cd-dd407302452f'), country='Malawi', item_type='Meat', order_date='3/1/2016', order_priority='H', region='Sub-Saharan Africa', sales_channel='Offline', ship_date='3/12/2016', total_cost=Decimal('1857730.86'), total_profit=Decimal('291376.8'), total_revenue=Decimal('2149107.66'), unit_cost=Decimal('364.69'), unit_price=Decimal('421.89'), units_sold=5094)
Row(order_id=UUID('c96fc13d-24eb-4a06-9957-78a4a64fdfdc'), country='New Zealand', item_type='Beverages', order_date='10/11/2012', order_priority='M', region='Australia and Oceania', sales_channel='Online', ship_date='11/4/2012', total_cost=Decimal('184000.52'), total_profit=Decimal('90640.08'), total_revenue=Decimal('274640.6'), unit_cost=Decimal('31.79'), unit_price=Decimal('47.45'), units_sold=5788)
Row(order_id=UUID('d016c4ed-ff45-47c2-ac2b-3e984f922b07'), country='Greece', item_type='Cereal', order_date='8/22/2015', order_priority='H', region='Europe', sales_ch

In [47]:
# Create Silver Level Table
session.execute("""
    CREATE TABLE IF NOT EXISTS silver_table (
        order_id UUID PRIMARY KEY,
        region TEXT,
        country TEXT,
        item_type TEXT,
        sales_channel TEXT,
        total_revenue DECIMAL,
    );
""")
print("Silver Level Table created successfully.")



Silver Level Table created successfully.


In [48]:
# Cleaning the data before inserting into the silver table
# Drop rows that have null or missing values and only picking useful columns to the silver table.
df_cleaned = df.dropna(subset=['Region', 'Country', 'Item Type', 'Sales Channel', 'Order Date', 'Ship Date',
                               'UnitsSold', 'UnitPrice', 'UnitCost', 'TotalRevenue', 'TotalCost', 'TotalProfit'])

# Insert cleaned data from bronze table into the Silver table
for _, row in df_cleaned.iterrows():
    session.execute("""
        INSERT INTO silver_table (order_id, region, country, item_type, sales_channel, total_revenue)
        VALUES (%s, %s, %s, %s, %s, %s)
    """, (
        uuid.uuid4(),  # Generate UUID for each order
        row['Region'], row['Country'], row['Item Type'], row['Sales Channel'], row['TotalRevenue']
    ))
print("Data inserted into silver_table.")
data = session.execute("SELECT * FROM silver_table LIMIT 5;")
for row in data:
    print(row)

Data inserted into silver_table.
Row(order_id=UUID('9c3b8cd2-2d0b-437c-9437-a6ba85d2df1e'), country='Iceland', item_type='Baby Food', region='Europe', sales_channel='Offline', total_revenue=Decimal('628499.36'))
Row(order_id=UUID('76c4dc1a-170a-494c-93d1-f775641d61d4'), country='Thailand', item_type='Fruits', region='Asia', sales_channel='Offline', total_revenue=Decimal('14508.15'))
Row(order_id=UUID('f9f54ecc-7942-4558-a2e8-200c5aac5d54'), country='Oman', item_type='Cosmetics', region='Middle East and North Africa', sales_channel='Online', total_revenue=Decimal('3470056.4'))
Row(order_id=UUID('440d2f8d-c160-414b-bffc-cd1b99d101d9'), country='Indonesia', item_type='Household', region='Asia', sales_channel='Online', total_revenue=Decimal('416332.21'))
Row(order_id=UUID('c06e1bcf-caa4-41ce-b819-e223a26a1fcf'), country='Antigua and Barbuda ', item_type='Office Supplies', region='Central America and the Caribbean', sales_channel='Offline', total_revenue=Decimal('4100669.37'))


In [49]:
# Create Gold Level Table 1: Sales by country
session.execute("""
    CREATE TABLE IF NOT EXISTS gold_sales_by_country (
        country TEXT PRIMARY KEY,
        total_sales DECIMAL
    );
""")
print("Gold Level Table (Sales by Country) created successfully.")

# Create Gold Level Table 2: Sales by Item Type
session.execute("""
    CREATE TABLE IF NOT EXISTS gold_sales_by_item_type (
        item_type TEXT PRIMARY KEY,
        total_sales DECIMAL
    );
""")
print("Gold Level Table (Sales by Item Type) created successfully.")

# Create Gold Level Table 3: Sales by region
session.execute("""
    CREATE TABLE IF NOT EXISTS gold_sales_by_region (
        region TEXT PRIMARY KEY,
        total_sales DECIMAL
    );
""")
print("Gold Level Table (Sales by region) created successfully.")

Gold Level Table (Sales by Country) created successfully.
Gold Level Table (Sales by Item Type) created successfully.
Gold Level Table (Sales by region) created successfully.


In [50]:
rows_silver_table = session.execute("SELECT * FROM silver_table;")

region_sales = {}
country_sales = {}
item_type_sales = {}

for row in rows_silver_table:
    # Aggregate by Region
    if row.region in region_sales:
        region_sales[row.region] += row.total_revenue
    else:
        region_sales[row.region] = row.total_revenue

    # Aggregate by Item Type
    if row.country in country_sales:
        country_sales[row.country] += row.total_revenue
    else:
        country_sales[row.country] = row.total_revenue

    # Aggregate by Item Type
    if row.item_type in item_type_sales:
        item_type_sales[row.item_type] += row.total_revenue
    else:
        item_type_sales[row.item_type] = row.total_revenue

# Insert aggregated data into Gold Level Table 1: Sales by Region
for country, total_sales in country_sales.items():
    session.execute("""
        INSERT INTO gold_sales_by_country (country, total_sales)
        VALUES (%s, %s)
    """, (country, total_sales))

# Insert aggregated data into Gold Level Table 2: Sales by Item Type
for item_type, total_sales in item_type_sales.items():
    session.execute("""
        INSERT INTO gold_sales_by_item_type (item_type, total_sales)
        VALUES (%s, %s)
    """, (item_type, total_sales))

# Insert aggregated data into Gold Level Table 3: Sales by Channel
for region, total_sales in region_sales.items():
    session.execute("""
        INSERT INTO gold_sales_by_region (region, total_sales)
        VALUES (%s, %s)
    """, (region, total_sales))

print("Data inserted into gold tables")


Data inserted into gold tables


In [51]:
# Querying the table gold_sales_by_country table to verify
rows = session.execute("SELECT * FROM gold_sales_by_country;")
for row in rows:
    print(row)



Row(country='Malaysia', total_sales=Decimal('434357.3'))
Row(country='Israel', total_sales=Decimal('223442.05'))
Row(country='Serbia', total_sales=Decimal('802989.44'))
Row(country='Djibouti', total_sales=Decimal('61415.36'))
Row(country='Egypt', total_sales=Decimal('130261.76'))
Row(country='Nicaragua', total_sales=Decimal('5944506.05'))
Row(country='Romania', total_sales=Decimal('1726589.35'))
Row(country='Sao Tome and Principe', total_sales=Decimal('301612.8'))
Row(country='Liberia', total_sales=Decimal('337990.72'))
Row(country='Vanuatu', total_sales=Decimal('1222089.25'))
Row(country='Morocco', total_sales=Decimal('503890.08'))
Row(country='Lebanon', total_sales=Decimal('3699975.3'))
Row(country='Madagascar', total_sales=Decimal('339860.8'))
Row(country='Mali', total_sales=Decimal('2884921.59'))
Row(country='Togo', total_sales=Decimal('2101183.2'))
Row(country='Sri Lanka', total_sales=Decimal('12866.07'))
Row(country='Canada', total_sales=Decimal('3263260.8'))
Row(country='Vietnam

In [52]:
# Querying the table gold_sales_by_item_type table to verify
rows = session.execute("SELECT * FROM gold_sales_by_item_type;")
for row in rows:
    print(row)



Row(item_type='Household', total_sales=Decimal('38519082.8'))
Row(item_type='Office Supplies', total_sales=Decimal('27880904.94'))
Row(item_type='Vegetables', total_sales=Decimal('1135114.08'))
Row(item_type='Snacks', total_sales=Decimal('2193642.66'))
Row(item_type='Personal Care', total_sales=Decimal('3191147.85'))
Row(item_type='Meat', total_sales=Decimal('21278865.93'))
Row(item_type='Fruits', total_sales=Decimal('615033.6'))
Row(item_type='Beverages', total_sales=Decimal('2145024.7'))
Row(item_type='Cereal', total_sales=Decimal('9416123.2'))
Row(item_type='Cosmetics', total_sales=Decimal('28727100.4'))
Row(item_type='Baby Food', total_sales=Decimal('5200564.16'))
Row(item_type='Clothes', total_sales=Decimal('4387373.44'))


In [53]:
# Querying the table gold_sales_by_region table to verify
rows = session.execute("SELECT * FROM gold_sales_by_region;")
for row in rows:
    print(row)

Row(region='Australia and Oceania', total_sales=Decimal('10711258.13'))
Row(region='Europe', total_sales=Decimal('34964749.83'))
Row(region='Middle East and North Africa', total_sales=Decimal('24765127.25'))
Row(region='Central America and the Caribbean', total_sales=Decimal('17570835.42'))
Row(region='Asia', total_sales=Decimal('28840812.19'))
Row(region='Sub-Saharan Africa', total_sales=Decimal('24225437.42'))
Row(region='North America', total_sales=Decimal('3611757.52'))
